In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats
import seaborn as sns

Matplotlib is building the font cache; this may take a moment.


In [7]:
from sklearn.datasets import fetch_california_housing
ca=fetch_california_housing()

In [8]:
dir(ca)

['DESCR', 'data', 'feature_names', 'frame', 'target', 'target_names']

In [15]:
ca

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [10]:
x=ca.data
y=ca.target

In [21]:
x.shape

(20640, 8)

In [22]:
y.shape

(20640,)

In [ ]:
x 

In [13]:
data=np.concatenate((x,y.reshape(-1,1)),axis=1)

In [23]:
data.shape

(20640, 9)

In [19]:
df=pd.DataFrame(data,columns=ca.feature_names+['target'])

In [110]:
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,3.422
...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,0.847


In [26]:
df.drop(['Latitude','Longitude'],axis=1,inplace=True)

In [111]:
df_trim=df.copy()

In [38]:
a=df['HouseAge'].quantile(0.25)

In [112]:
def trim_outliner (df,cols,factor):
    for col in cols:
        drop_outliner=np.array([])
        q1=df[col].quantile(0.25)
        q3=df[col].quantile(0.75)
        iqr=q3-q1
        s=df[col]
        lower_whisker=q3-(factor*iqr)
        upper_whisker=q1+(factor*iqr)
        indexes=s[(s<lower_whisker) | (s> upper_whisker)].index
        drop_outliner=np.append(drop_outliner,indexes)
    return drop_outliner   

In [45]:
indexes=s[(s<lower_whisker) | (s> upper_whisker)].index

NameError: name 's' is not defined

In [113]:
features= ('MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup',
       'target')

(787.0, 1725.0, 938.0, 318.0, 2194.0)

In [114]:
dropped=np.unique(trim_outliner(df_trim,features,1.5))

In [115]:
df_trim.drop(dropped,inplace=True)

In [116]:
df_trim.shape

(17563, 7)

In [98]:
df.shape

(20640, 7)

In [117]:
20640-17563

3077

In [118]:
20640/17563

1.1751978591356829

In [ ]:
#Capping

In [119]:
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,3.422
...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,0.847


In [122]:
df_cap=df.copy()

In [121]:
def cap_outliner(df,cols,from_low_end,from_upper_end):
    for col in cols:
        # low_bound=df[col].quantile(from_low_end)
        #upper_bound=df[col].quantile(from_upper_end)
       # df[col]=np.where(df[col]>upper_bound,upper-bound, np.where(df[col]<lower_bound,lower_bound,df[col])        
       
        stats.mstats.winsorize(a=df[col],limits=(from_low_end,from_upper_end),inplace=True)

In [123]:
cap_outliner(df_cap,features,0.01,0.01)

In [124]:
df_cap.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,target
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.852245,28.648692,5.334211,1.077082,1404.023692,2.916378,2.069232
std,1.804005,12.566882,1.321745,0.158560,972.980477,0.732341,1.152996
min,1.068500,4.000000,2.581006,0.872727,88.000000,1.536605,0.500000
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,2.647250
max,10.598100,52.000000,10.359649,2.127753,5807.000000,5.396000,5.000010


In [125]:
df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,target
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,5.000010
